In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/db'
!ls

/content/drive/My Drive/Colab Notebooks/db
API_AG.LND.AGRI.K2_DS2_en_csv_v2_716226.csv	Meteorite_Landings.csv
API_AG.LND.FRST.K2_DS2_en_csv_v2_716262.csv	poblacion.csv
API_EG.USE.ELEC.KH.PC_DS2_en_csv_v2_715482.csv	test.csv
API_EN.ATM.CO2E.KT_DS2_en_csv_v2_713263.csv	test.hdf
API_SP.POP.TOTL_DS2_en_csv_v2_713131.csv	test.json
covid_19_data.csv				test.parquet
flights.db					test.pkl
GlobalLandTemperaturesByCountry.csv		test.xlsx
london_merged.csv


In [ ]:
import pandas as pd
import numpy as np
pd.__version__

'1.1.5'

Traigo los datos de poblacion.csv y los guardo en un dataframe:

Tengo los datos: Pais, Año, Poblacion

In [ ]:
df_pob = pd.read_csv('poblacion.csv')
df_pob

,Country,year,pop
0,Afghanistan,2015,"34,413,603.0"
1,Albania,2015,"2,880,703.0"
2,Algeria,2015,"39,728,025.0"
3,American Samoa,2015,"55,812.0"
4,Andorra,2015,"78,011.0"
...,...,...,...
1035,Pre-demographic dividend,2018,"919,485,393.0"
1036,Small states,2018,"40,575,321.0"
1037,South Asia,2018,"1,814,388,744.0"
1038,South Asia (IDA & IBRD),2018,"1,814,388,744.0"


Como poblacion lo tengo en un formato científico, para tener una mejor visializacion, especifico un formato que quiero con '{:,.1f}'.format

In [ ]:
pd.options.display.float_format = '{:,.1f}'.format

Convierto el año a una variable categorica para el uso de los multiples indices .apply(str))

In [ ]:
df_pob['year'] = pd.Categorical(df_pob['year'].apply(str))

Miro los tipos de variables que tengo:

In [ ]:
df_pob.dtypes

Country      object
year       category
pop         float64
dtype: object

Creo un filtro para hacer una base mas pequeña y escogo solo dos paises de country con una funcion .isin que crea un vector de variables booleanas(falso y verdadero) que me permitira filtrar algunos paises

In [ ]:
idx_filtro = df_pob['Country']. isin(['Aruba', 'Colombia']) # Falso para todos los paises menos para aruba y colombia  
idx_filtro

0       False
1       False
2       False
3       False
4       False
        ...  
1035    False
1036    False
1037    False
1038    False
1039    False
Name: Country, Length: 1040, dtype: bool

Nuevo df para mostrar los dos paises verdaderos con los años respectivos de poblacion (del 2015 al 2018):

In [ ]:
df_sample = df_pob[idx_filtro]
df_sample

,Country,year,pop
9,Aruba,2015,"104,341.0"
42,Colombia,2015,"47,520,667.0"
269,Aruba,2016,"104,872.0"
302,Colombia,2016,"48,171,392.0"
529,Aruba,2017,"105,366.0"
562,Colombia,2017,"48,901,066.0"
789,Aruba,2018,"105,845.0"
822,Colombia,2018,"49,648,685.0"


## `Los multiples indices permiten extructurar los datos para un mejor analisis`

Crear multiples indices:

Dentro los parentesis coloco las columnas que voy a volver indice

Puedo visualizar que el pais aruba tiene sus datos adjuntos y separados del pais colombia y viseversa

In [ ]:
df_sample = df_sample.set_index(['Country','year']).sort_index()
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

Ahora si solo quiero seleccionar colombia:

Obtengo solo el pequeño df de colombia(año y poblacion)

In [ ]:
df_sample.loc['Colombia',:]

,pop
year,
2015,"47,520,667.0"
2016,"48,171,392.0"
2017,"48,901,066.0"
2018,"49,648,685.0"


Si solo quiero datos de colombia, de un año en especifico:

In [ ]:
df_sample.loc['Colombia',:].loc['2016',:]

pop   48,171,392.0
Name: 2016, dtype: float64

Para seleccionar multiples indices con la funcion xs:

In [ ]:
df_sample.xs(['Aruba'])

,pop
year,
2015,"104,341.0"
2016,"104,872.0"
2017,"105,366.0"
2018,"105,845.0"


In [ ]:
df_sample.xs(['Aruba', '2018'])

pop   105,845.0
Name: (Aruba, 2018), dtype: float64

Para concentrarme solo en el año de ambos paises:

In [ ]:
df_sample.xs('2018', level='year')

,pop
Country,
Aruba,"105,845.0"
Colombia,"49,648,685.0"


Aplicamos esto a todo nuestro conjunto de datos:

Todo el df esta organizado con los indices country and year

Con ascending = [True,True] organizo los datos alfabetico

Con ascending = [False,True] organizo los datos alfabetico inverso(comienzan en Z y terminan en A)

In [ ]:
df_countries = df_pob.set_index(['Country','year']).sort_index(ascending = [True,True])
df_countries

pop
Country     year             
Afghanistan 2015 34,413,603.0
            2016 35,383,128.0
            2017 36,296,400.0
            2018 37,172,386.0
Albania     2015  2,880,703.0
...                       ...
Zambia      2018 17,351,822.0
Zimbabwe    2015 13,814,629.0
            2016 14,030,390.0
            2017 14,236,745.0
            2018 14,439,018.0

[1040 rows x 1 columns]

con .IndexSlice obtengo un pedazo del df de los paises Aruba hasta Austria y con los años del 2015 al 2017:

In [ ]:
ids = pd.IndexSlice
df_countries.loc[ids['Aruba':'Austria','2015':'2017'],:].sort_index()

pop
Country   year             
Aruba     2015    104,341.0
          2016    104,872.0
          2017    105,366.0
Australia 2015 23,815,995.0
          2016 24,190,907.0
          2017 24,601,860.0
Austria   2015  8,642,699.0
          2016  8,736,668.0
          2017  8,797,566.0

Obtengo los indices del primer nivel de df_countries:

In [ ]:
df_countries.index

MultiIndex([('Afghanistan', '2015'),
            ('Afghanistan', '2016'),
            ('Afghanistan', '2017'),
            ('Afghanistan', '2018'),
            (    'Albania', '2015'),
            (    'Albania', '2016'),
            (    'Albania', '2017'),
            (    'Albania', '2018'),
            (    'Algeria', '2015'),
            (    'Algeria', '2016'),
            ...
            ('Yemen, Rep.', '2017'),
            ('Yemen, Rep.', '2018'),
            (     'Zambia', '2015'),
            (     'Zambia', '2016'),
            (     'Zambia', '2017'),
            (     'Zambia', '2018'),
            (   'Zimbabwe', '2015'),
            (   'Zimbabwe', '2016'),
            (   'Zimbabwe', '2017'),
            (   'Zimbabwe', '2018')],
           names=['Country', 'year'], length=1040)

Para obtener los datos del primer indice(paises) con .get_level_values(0)

Con .get_level_values(1) obtengo los datos de year



In [ ]:
df_countries.index.get_level_values(0)

Index(['Afghanistan', 'Afghanistan', 'Afghanistan', 'Afghanistan', 'Albania',
       'Albania', 'Albania', 'Albania', 'Algeria', 'Algeria',
       ...
       'Yemen, Rep.', 'Yemen, Rep.', 'Zambia', 'Zambia', 'Zambia', 'Zambia',
       'Zimbabwe', 'Zimbabwe', 'Zimbabwe', 'Zimbabwe'],
      dtype='object', name='Country', length=1040)

In [ ]:
df_countries.index.get_level_values(1)

CategoricalIndex(['2015', '2016', '2017', '2018', '2015', '2016', '2017',
                  '2018', '2015', '2016',
                  ...
                  '2017', '2018', '2015', '2016', '2017', '2018', '2015',
                  '2016', '2017', '2018'],
                 categories=['2015', '2016', '2017', '2018'], ordered=False, name='year', dtype='category', length=1040)

Para seleccionar datos de un df tambien lo podemos hacer llamando sus columnas

Lo que obtengo es una serie que tiene dos indices

In [ ]:
df_countries['pop']

Country      year
Afghanistan  2015   34,413,603.0
             2016   35,383,128.0
             2017   36,296,400.0
             2018   37,172,386.0
Albania      2015    2,880,703.0
                        ...     
Zambia       2018   17,351,822.0
Zimbabwe     2015   13,814,629.0
             2016   14,030,390.0
             2017   14,236,745.0
             2018   14,439,018.0
Name: pop, Length: 1040, dtype: float64

Puedo seleccionar un pais y un año en especifico

In [ ]:
df_countries['pop']['Colombia']['2018']

49648685.0

## `Cuando trabajamos con un df con multiples indices y queremos extraer un dato en especifico, lo podemos hacer a travez de una cadena de instrucciones`

## La gran utilidad de un sistema de multiples indices es cuando aplicamos funciones matematicas sobre el

Hago una suma de la poblacion en el nivel de indice del año

In [ ]:
df_countries.sum(level='year')

,pop
year,
2015,"65,679,147,019.0"
2016,"66,487,930,677.0"
2017,"67,294,176,701.0"
2018,"68,087,886,692.0"


Para visualizar distinto el df

In [ ]:
df_sample

pop
Country  year             
Aruba    2015    104,341.0
         2016    104,872.0
         2017    105,366.0
         2018    105,845.0
Colombia 2015 47,520,667.0
         2016 48,171,392.0
         2017 48,901,066.0
         2018 49,648,685.0

Convierto las filas en columnas y viceversa (paises vs año)

In [ ]:
df_sample.unstack('year')

pop                                       
year             2015         2016         2017         2018
Country                                                     
Aruba       104,341.0    104,872.0    105,366.0    105,845.0
Colombia 47,520,667.0 48,171,392.0 48,901,066.0 49,648,685.0

In [ ]:
df_sample.unstack('Country')

pop             
Country     Aruba     Colombia
year                          
2015    104,341.0 47,520,667.0
2016    104,872.0 48,171,392.0
2017    105,366.0 48,901,066.0
2018    105,845.0 49,648,685.0